### **랜덤으로 이미지 뽑아내기** 

In [3]:
# 랜덤으로 이미지 뽑아내기 
import os
import shutil
import random

def move_random_images(src_folder, dst_folder, n=100, exts=(".jpg", ".png", ".jpeg")):
    os.makedirs(dst_folder, exist_ok=True)

    # 이미지 목록 가져오기
    image_files = [f for f in os.listdir(src_folder) if f.lower().endswith(exts)]
    if n > len(image_files):
        raise ValueError(f"요청한 {n}개가 폴더 내 이미지 수 {len(image_files)}보다 많습니다.")

    # 랜덤 샘플링
    selected_files = random.sample(image_files, n)

    # 복사
    for file_name in selected_files:
        src_path = os.path.join(src_folder, file_name)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.copy(src_path, dst_path)

    print(f"총 {n}개의 이미지가 {dst_folder}로 복사되었습니다.")

# 사용 예시
move_random_images(r"F:\2.프로젝트\파나시아\2025\[KOLAS] 공인 시험\MODEL5\total\img", 
                    r"F:\2.프로젝트\파나시아\2025\[KOLAS] 공인 시험\MODEL5\test_img", n=50)


총 50개의 이미지가 F:\2.프로젝트\파나시아\2025\[KOLAS] 공인 시험\MODEL5\test_img로 복사되었습니다.


### **이미지와 라벨 매칭 (불일치 파일 제외)**

In [ ]:
# 이미지 옮기기 
import shutil,os 

img = os.listdir("F:/2.프로젝트/파나시아/2025/model1/dataset/train/images/")
label = os.listdir('F:/2.프로젝트/파나시아/2025/model1/dataset/train/labels/')

label_,image_ = list(),list()
for i in label:
    label_.append(i.split(".")[0])

for i in img:
    image_.append(i.split(".")[0])

for i in set(label_) - set(image_):
    shutil.move("F:/2.프로젝트/파나시아/2025/model1/dataset/train3/images/"+i+".jpg", "F:/2.프로젝트/파나시아/2025/model1/dataset/train3/제외/images/"+i+".jpg")

In [17]:
img = os.listdir("F:/2.프로젝트/파나시아/2025/model1/dataset/train3/제외/images/")
label = os.listdir('F:/2.프로젝트/파나시아/2025/model1/dataset/train3/제외/labels/')

label_,image_ = list(),list()
for i in label:
    label_.append(i.split(".")[0])

for i in img:
    image_.append(i.split(".")[0])

In [18]:
set(label_) - set(image_)
for i in set(label_) - set(image_):
    shutil.move("F:/2.프로젝트/파나시아/2025/model1/dataset/train3/images/"+i+".jpg", "F:/2.프로젝트/파나시아/2025/model1/dataset/train3/제외/images/"+i+".jpg")

### **고정좌표 마스킹**

In [8]:
# 고정좌표 마스킹 
import os
import cv2

# 디렉토리 설정
image_dir = "C:/Users/user/Desktop/test/MODEL3/image"      # 원본 이미지 폴더
output_dir = "C:/Users/user/Desktop/test/MODEL3/fixed_crop"
os.makedirs(output_dir, exist_ok=True)

# 고정된 crop 좌표 (픽셀 기준)
# 예: 왼쪽 상단 (x1, y1), 오른쪽 하단 (x2, y2)
cx, cy, bw, bh = map(float, [0.03715, 0.13296, 0.06835, 0.22894])
w,h = 1920,1080
x1 = int((cx - bw / 2) * w)
y1 = int((cy - bh / 2) * h)
x2 = int((cx + bw / 2) * w)
y2 = int((cy + bh / 2) * h)

# 경계 클리핑
x1 = max(0, x1)
y1 = max(0, y1)
x2 = min(w, x2)
y2 = min(h, y2)

for img_name in os.listdir(image_dir):
    if not img_name.endswith(".jpg"):
        continue

    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    h, w = img.shape[:2]

    img[y1:y2, x1:x2] = (0, 0, 0)

    # 저장
    out_path = os.path.join(output_dir, img_name)
    cv2.imwrite(out_path, img)


In [6]:
cx, cy, bw, bh = map(float, [0.03715, 0.13296, 0.06835, 0.22894])

x1 = int((cx - bw / 2) * w)
y1 = int((cy - bh / 2) * h)
x2 = int((cx + bw / 2) * w)
y2 = int((cy + bh / 2) * h)

# 경계 클리핑
x1 = max(0, x1)
y1 = max(0, y1)
x2 = min(w, x2)
y2 = min(h, y2)

### **클래스별 Crop Augmentation (마스킹)**

In [ ]:
# class 0인 영역 검정색 마스킹 - crop augmentation
import os
import cv2
import numpy as np

# 디렉토리 설정
image_dir =  "C:/Users/user/Desktop/test/MODEL3/image"       # 원본 이미지 폴더
label_dir ="C:/Users/user/Desktop/test/MODEL3/label"       # YOLO .txt 라벨 폴더
output_dir = "C:/Users/user/Desktop/test/MODEL3/crops_class0"
os.makedirs(output_dir, exist_ok=True)

x1, y1, x2, y2 = 100, 100, 300, 300
for img_name in os.listdir(image_dir):
    if not img_name.endswith(".jpg"):
        continue

    img_path = os.path.join(image_dir, img_name)
    label_path = os.path.join(label_dir, img_name.replace(".jpg", ".txt"))

    img = cv2.imread(img_path)
    if img is None or not os.path.exists(label_path):
        continue

    h, w = img.shape[:2]

    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls_id = int(parts[0])
            if cls_id != 0:  # ❗ 클래스 0과 1만 마스킹
                continue  # ❗ 클래스 0만 마스킹

            cx, cy, bw, bh = map(float, parts[1:])

            # YOLO bbox → 픽셀 좌표로 변환
            x1 = int((cx - bw / 2) * w)
            y1 = int((cy - bh / 2) * h)
            x2 = int((cx + bw / 2) * w)
            y2 = int((cy + bh / 2) * h)

            # 경계 클리핑
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(w, x2)
            y2 = min(h, y2)

            # 해당 bbox를 검정색으로 덮기
            img[y1:y2, x1:x2] = (0, 0, 0)

    # 저장
    out_path = os.path.join(output_dir, img_name)
    cv2.imwrite(out_path, img)

### **원하는 클래스 삭제**

In [9]:
# 원하는 class 삭제하는 코드 
import os

# YOLO 라벨이 저장된 디렉토리
label_dir ="C:/Users/user/Desktop/test/MODEL3/label"       # YOLO .txt 라벨 폴더

# 삭제할 클래스 ID
remove_classes = {0}

# 모든 .txt 라벨 파일 순회
for file_name in os.listdir(label_dir):
    if not file_name.endswith(".txt"):
        continue

    file_path = os.path.join(label_dir, file_name)

    # 유효한 라인만 남김
    with open(file_path, 'r') as f:
        lines = f.readlines()

    filtered_lines = []
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        cls_id = int(parts[0])
        if cls_id not in remove_classes:
            filtered_lines.append(line)

    # 덮어쓰기 (없으면 빈 파일이 됨)
    with open(file_path, 'w') as f:
        f.writelines(filtered_lines)


### **데이터셋 분할 (Train/Valid 분리)**

In [11]:
# dataset split
import os
import random
import shutil

# 설정값
train_img_dir = "F:/2.프로젝트/파나시아/2025/model3/dataset/train/images"
train_lbl_dir = "F:/2.프로젝트/파나시아/2025/model3/dataset/train/labels"
test_img_dir = "F:/2.프로젝트/파나시아/2025/model13/dataset/valid/images"
test_lbl_dir = "F:/2.프로젝트/파나시아/2025/model13/dataset/valid/labels"

test_ratio = 0.05  # 전체 중 몇 %를 테스트셋으로 할지
random_seed = 42  # 재현 가능성

# 디렉토리 생성
os.makedirs(test_img_dir, exist_ok=True)
os.makedirs(test_lbl_dir, exist_ok=True)

# train 이미지 파일 리스트 (확장자 고려)
image_files = [f for f in os.listdir(train_img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
random.seed(random_seed)
test_files = random.sample(image_files, int(len(image_files) * test_ratio))

for img_file in test_files:
    # 파일명에서 확장자 제거
    base_name = os.path.splitext(img_file)[0]
    lbl_file = base_name + ".txt"

    # 파일 이동
    shutil.move(os.path.join(train_img_dir, img_file), os.path.join(test_img_dir, img_file))
    shutil.move(os.path.join(train_lbl_dir, lbl_file), os.path.join(test_lbl_dir, lbl_file))

print(f"✅ {len(test_files)}개의 이미지가 test 셋으로 이동되었습니다.")


✅ 144개의 이미지가 test 셋으로 이동되었습니다.


In [ ]:
from ultralytics import YOLO  # YOLOv8/YOLOv11 계열
import os 

model = YOLO("F:/2.프로젝트/파나시아/2025/model5/model5_0428.pt")  # load a pretrained model (recommended for training)

# 추론 시 클래스 1만 대상으로 예측

if __name__ == '__main__':
    # Load a model
    #image_path = 'F:/2.프로젝트/파나시아/2025/model2/CAM0017_20250331_131130_forklift/'
    image_path = "F:/2.프로젝트/파나시아/2025/model5/dataset/test/images/"
    

    for i in os.listdir(image_path):
        image_file = image_path + i
        
        if '.jpg' in image_file:
            print('image_file: ', image_file)            
   
        # Predict
        results = model(image_file, save = True, save_txt=True,conf=0.8, iou=0.5)
        res_plot = results[0].plot()  

### **라벨 파일에 박스 추가**

In [ ]:
for label in lb:
    labels = """4 0.17484 0.01447 0.00776 0.02688
"""


    with open( 'F:/2.프로젝트/파나시아/2025/model4/data/DATA5/labels/' + label[:-4]+'.txt', "w", encoding="utf-8") as file:
         file.writelines(labels)

In [4]:
for label in lb:
    # 기존 파일 경로
    file_path = 'C:/Users/user/Desktop/test/MODEL1/전체 검수/v/' + label[:-4] + '.txt'
    
    # 기존 내용 읽기
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            existing_content = file.read()
    except FileNotFoundError:
        existing_content = ""  # 파일이 없으면 빈 문자열

    # 추가할 값
    new_value = "4 0.17484 0.01447 0.00776 0.02688\n"

    # 기존 내용 + 추가 값
    updated_content = existing_content + new_value

    # 파일에 다시 저장
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(updated_content)

### **class number 수정**

In [70]:
import os

def change_yolo_class_number(labels_dir, old_class_number, new_class_number):
    """
    YOLO 라벨 파일(.txt)에서 특정 클래스 번호를 다른 번호로 변경하고 파일을 업데이트합니다.

    Args:
        labels_dir (str): YOLO 라벨 파일들이 있는 디렉토리 경로.
        old_class_number (int): 변경하려는 기존 클래스 번호.
        new_class_number (int): 새로 변경할 클래스 번호.
    """
    if not os.path.isdir(labels_dir):
        print(f"오류: 지정된 경로 '{labels_dir}'는 유효한 디렉토리가 아닙니다.")
        return

    print(f"'{labels_dir}' 경로의 라벨 파일들을 분석하며 클래스 {old_class_number}를 {new_class_number}로 변경 중...")

    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_dir, filename)
            
            modified_lines = []
            file_modified = False # 파일이 수정되었는지 여부를 추적

            try:
                with open(filepath, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if parts: # 빈 줄이 아닌 경우
                            try:
                                class_number_str = parts[0]
                                current_class_number = int(class_number_str)

                                if current_class_number == old_class_number:
                                    # 클래스 번호 변경
                                    parts[0] = str(new_class_number)
                                    modified_lines.append(" ".join(parts) + "\n")
                                    file_modified = True
                                else:
                                    modified_lines.append(line)
                            except ValueError:
                                # 첫 번째 부분이 숫자가 아닌 경우 (예: 잘못된 형식) 해당 줄은 유지
                                modified_lines.append(line)
                        else:
                            # 빈 줄은 유지
                            modified_lines.append(line)
            except Exception as e:
                print(f"파일 읽기 오류: {filepath} - {e}")
                continue

            if file_modified:
                try:
                    # 수정된 내용으로 파일 다시 쓰기
                    with open(filepath, 'w') as f:
                        f.writelines(modified_lines)
                    print(f"'{filename}' 파일에서 클래스 {old_class_number}를 {new_class_number}로 변경했습니다.")
                except Exception as e:
                    print(f"파일 쓰기 오류: {filepath} - {e}")
            # else:
            #     print(f"'{filename}' 파일에는 변경할 클래스 {old_class_number}가 없습니다.")


if __name__ == "__main__":
    # TODO: 여기에 YOLO 라벨 파일들이 있는 실제 경로를 넣어주세요.
    # 예시: labels_directory = "C:/Users/user/Desktop/yolo_dataset/labels"
    labels_directory = "./dataset/test3/labels/" 
    
    # 변경하려는 기존 클래스 번호
    old_class = 4
    # 새로 변경할 클래스 번호
    new_class = 2

    if os.path.exists(labels_directory):
        change_yolo_class_number(labels_directory, old_class, new_class)
        print("모든 라벨 파일의 클래스 번호 변경 처리가 완료되었습니다.")
    else:
        print(f"오류: 지정된 경로 '{labels_directory}'를 찾을 수 없습니다. 올바른 경로를 입력해주세요.")

'./dataset/test3/labels/' 경로의 라벨 파일들을 분석하며 클래스 4를 2로 변경 중...
'cam72 _0417_000.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_030.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_060.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_061.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_090.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_091.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1020.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1021.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1050.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1051.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1080.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1081.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1110.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1111.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1140.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1141.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1170.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1171.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_120.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1200.txt' 파일에서 클래스 4를 2로 변경했습니다.
'cam72 _0417_1201.txt' 파일에

### **좌표 고정 복붙**

In [ ]:
for label in lb:
    with open(path + label, "r", encoding="utf-8") as file:
        lines = file.readlines()  #  모든 줄을 리스트로 읽음
        final = list()
        for line in lines:
            line_f = line.split('\n')
            line_c = list(map(float, line_f[0].split(' ')))
            if line_c[0] == 2:
                line_c[0] = int(3)
                final.append(' '.join(map(str,line_c))+'\n')
            # elif line_c[0] == 1:
            #     line_c[0] = int(2)
            #     final.append(' '.join(map(str,line_c))+'\n')
            # elif line_c[0] == 2:
            #     line_c[0] = int(0)
            #     final.append(' '.join(map(str,line_c))+'\n')

    print(final)


    with open(path + label, "w", encoding="utf-8") as file:
        file.writelines(final)

### **라벨 파일 복사**

In [ ]:
# 박스 하나만 추가 
model2 = os.listdir('F:/2.프로젝트/파나시아/2025/model4/data/data2/labels/')
len(model2)

ff= ['0 0.78241 0.34479 0.13424 0.30531\n','0 0.88512 0.34819 0.11991 0.25266\n']


for txt in model2:
    with open('F:/2.프로젝트/파나시아/2025/model4/data/data2/labels/' + txt, "r", encoding="utf-8") as file1:
        lines1 = file1.readlines()
        final = list()
        final.extend(lines1)  
        final.extend(ff)


        print(final)

        with open('F:/2.프로젝트/파나시아/2025/model4/data/data2/labels/' + txt, "w", encoding="utf-8") as file:
            file.writelines(final)



### **이미지 생성** 

In [ ]:
import os
import cv2

# === 경로 설정 ===
label_dir = 'F:/2.프로젝트/파나시아/2025/model4/data/라벨링 예정 _1/label/'
image_dir = 'F:/2.프로젝트/파나시아/2025/model4/data/라벨링 예정 _1/image/'  # 이미지가 존재하는 폴더
output_dir = 'F:/2.프로젝트/파나시아/2025/model4/data/라벨링 예정 _1/output'  # 결과 저장 폴더
os.makedirs(output_dir, exist_ok=True)

# === 라벨 파일 순회 ===
for label_file in os.listdir(label_dir):
    if not label_file.endswith('.txt'):
        continue

    # === 이미지 이름과 경로 추정 ===
    base_name = os.path.splitext(label_file)[0]
    img_path = os.path.join(image_dir, base_name + '.jpg')  # 확장자 변경 가능
    label_path = os.path.join(label_dir, label_file)

    # === 이미지 불러오기 ===
    image = cv2.imread(img_path)
    if image is None:
        print(f"이미지 못 찾음: {img_path}")
        continue
    h_img, w_img = image.shape[:2]

    # === 라벨 파일 읽기 ===
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        _, x_center, y_center, width, height = map(float, parts)

        # YOLO 상대좌표 → 절대 pixel 좌표 변환
        x = int((x_center - width / 2) * w_img)
        y = int((y_center - height / 2) * h_img)
        w = int(width * w_img)
        h = int(height * h_img)

        # 이미지 경계 벗어나지 않게 처리
        x = max(0, x)
        y = max(0, y)
        w = min(w, w_img - x)
        h = min(h, h_img - y)

        # === 검정색으로 마스킹 처리 ===
        #image[y:y+h, x:x+w] = (0, 0, 0)

        # === Blur 처리 ===
        roi = image[y:y+h, x:x+w]
        blurred_roi = cv2.GaussianBlur(roi, (51, 51), 0)  # 커널 크기 (홀수)
        image[y:y+h, x:x+w] = blurred_roi

    # === 결과 이미지 저장 ===
    output_path = os.path.join(output_dir, base_name + '.jpg')
    cv2.imwrite(output_path, image)
    print(f"저장 완료: {output_path}")


In [20]:

import os

def update_yolo_labels(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory_path, filename)
            updated_lines = []
            with open(filepath, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if parts and parts[0] == '4':
                        parts[0] = '2'
                    updated_lines.append(' '.join(parts))

            with open(filepath, 'w') as f:
                f.write('\n'.join(updated_lines))
    print(f"'{directory_path}' 디렉토리 내 모든 YOLO 라벨 파일이 업데이트되었습니다.")

# 사용자 지정 디렉토리 경로
yolo_directory = './dataset/train3/labels/'

update_yolo_labels(yolo_directory)


'./dataset/train3/labels/' 디렉토리 내 모든 YOLO 라벨 파일이 업데이트되었습니다.


In [18]:
import os

def process_yolo_labels(labels_dir):
    """
    YOLO 라벨 파일(.txt)에서 클래스 번호 '2'인 라벨을 삭제하고 파일을 업데이트합니다.

    Args:
        labels_dir (str): YOLO 라벨 파일들이 있는 디렉토리 경로.
    """
    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_dir, filename)
            
            # 파일 내용 읽기
            with open(filepath, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts: # 빈 줄이 아닌 경우
                    try:
                        class_number = int(parts[0])
                        if class_number != 2:
                            new_lines.append(line)
                    except ValueError:
                        # 첫 번째 부분이 숫자가 아닌 경우 (예: 잘못된 형식) 해당 줄은 유지
                        new_lines.append(line)
                else:
                    # 빈 줄은 유지
                    new_lines.append(line)

            # 수정된 내용으로 파일 다시 쓰기
            with open(filepath, 'w') as f:
                f.writelines(new_lines)
            print(f"Processed: {filename}")

if __name__ == "__main__":
    # TODO: 여기에 YOLO 라벨 파일들이 있는 실제 경로를 넣어주세요.
    # 예시: labels_directory = "C:/Users/user/Desktop/yolo_dataset/labels"
    labels_directory = "./dataset/valid3/labels/" 
    
    if os.path.exists(labels_directory):
        process_yolo_labels(labels_directory)
        print("모든 라벨 파일 처리가 완료되었습니다.")
    else:
        print(f"오류: 지정된 경로 '{labels_directory}'를 찾을 수 없습니다. 올바른 경로를 입력해주세요.")


Processed: 20250512_10290.txt
Processed: 20250512_11100.txt
Processed: 20250512_11820.txt
Processed: 20250512_12840.txt
Processed: 20250512_5400.txt
Processed: 20250512_5461.txt
Processed: 20250512_5821.txt
Processed: 20250512_6060.txt
Processed: 20250512_6091.txt
Processed: 20250512_6241.txt
Processed: 20250512_6360.txt
Processed: 20250512_6570.txt
Processed: 20250512_6661.txt
Processed: 20250512_6780.txt
Processed: 20250512_6900.txt
Processed: 20250512_7230.txt
Processed: 20250512_7351.txt
Processed: 20250512_7380.txt
Processed: 20250512_7621.txt
Processed: 20250512_8520.txt
Processed: 20250512_9210.txt
Processed: cam02_41502_42691.txt
Processed: cam02_41502_42900.txt
Processed: cam02_41502_43020.txt
Processed: cam02_41502_43141.txt
Processed: cam02_41502_43200.txt
Processed: cam02_41502_43321.txt
Processed: cam02_41502_43410.txt
Processed: cam02_41502_43591.txt
Processed: cam02_41502_43681.txt
Processed: cam02_41502_43920.txt
Processed: cam02_41502_44070.txt
Processed: cam02_41502_4

In [1]:
import os

def remove_class_2_labels(labels_dir):
    """
    YOLO 라벨 파일(.txt)에서 클래스 번호 '2'인 라벨 줄을 삭제하고 파일을 업데이트합니다.

    Args:
        labels_dir (str): YOLO 라벨 파일들이 있는 디렉토리 경로.
    """
    if not os.path.isdir(labels_dir):
        print(f"오류: 지정된 경로 '{labels_dir}'는 유효한 디렉토리가 아닙니다.")
        return

    print(f"'{labels_dir}' 경로의 라벨 파일들에서 클래스 2 정보 삭제 중...")

    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_dir, filename)
            
            new_lines = []
            file_modified = False # 파일이 수정되었는지 여부를 추적

            try:
                # 파일 내용 읽기
                with open(filepath, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if parts: # 빈 줄이 아닌 경우
                            try:
                                class_number = int(parts[0])
                                if class_number != 2: # 클래스 번호가 2가 아닌 경우에만 추가
                                    new_lines.append(line)
                                else:
                                    file_modified = True # 클래스 2를 발견하여 삭제했으므로 파일 수정됨
                            except ValueError:
                                # 첫 번째 부분이 숫자가 아닌 경우 (예: 잘못된 형식) 해당 줄은 유지
                                new_lines.append(line)
                        else:
                            # 빈 줄은 유지
                            new_lines.append(line)
            except Exception as e:
                print(f"파일 읽기 오류: {filepath} - {e}")
                continue

            if file_modified:
                try:
                    # 수정된 내용으로 파일 다시 쓰기
                    with open(filepath, 'w') as f:
                        f.writelines(new_lines)
                    print(f"'{filename}' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.")
                except Exception as e:
                    print(f"파일 쓰기 오류: {filepath} - {e}")
            # else:
            #     print(f"'{filename}' 파일에는 삭제할 클래스 2 라벨 줄이 없습니다.")

if __name__ == "__main__":
    # TODO: 여기에 YOLO 라벨 파일들이 있는 실제 경로를 넣어주세요.
    # 예시: labels_directory = "C:/Users/user/Desktop/yolo_dataset/labels"
    labels_directory = "./dataset/test3/labels/" 
    
    if os.path.exists(labels_directory):
        remove_class_2_labels(labels_directory)
        print("모든 라벨 파일의 클래스 2 라벨 줄 삭제 처리가 완료되었습니다.")
    else:
        print(f"오류: 지정된 경로 '{labels_directory}'를 찾을 수 없습니다. 올바른 경로를 입력해주세요.")


'./dataset/test3/labels/' 경로의 라벨 파일들에서 클래스 2 정보 삭제 중...
'cam71_0416443_1021.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1022.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1026.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1032.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1033.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1034.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1035.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1037.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1038.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1044.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1046.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1048.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1049.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1050.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1054.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1055.txt' 파일에서 클래스 2 라벨 줄을 삭제하고 업데이트했습니다.
'cam71_0416443_1056.txt' 파일에서 클래

In [5]:
import os
import shutil

def move_labels_with_class_2(input_labels_dir, output_labels_dir):
    """
    클래스 번호 '2'를 포함하는 YOLO 라벨 파일(.txt)을 지정된 출력 디렉토리로 이동시킵니다.

    Args:
        input_labels_dir (str): YOLO 라벨 파일들이 있는 입력 디렉토리 경로.
        output_labels_dir (str): 클래스 번호 '2'를 포함하는 라벨 파일들이 이동될 디렉토리 경로.
    """
    if not os.path.exists(output_labels_dir):
        os.makedirs(output_labels_dir)
        print(f"출력 디렉토리를 생성했습니다: {output_labels_dir}")

    for filename in os.listdir(input_labels_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(input_labels_dir, filename)
            
            contains_class_2 = False
            try:
                with open(filepath, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if parts:
                            try:
                                class_number = int(parts[0])
                                if class_number == 2:
                                    contains_class_2 = True
                                    break  # 클래스 2를 찾았으니 더 이상 읽을 필요 없음
                            except ValueError:
                                # 첫 번째 부분이 숫자가 아닌 경우 무시
                                continue
            except Exception as e:
                print(f"파일 읽기 오류: {filepath} - {e}")
                continue

            if contains_class_2:
                try:
                    destination_path = os.path.join(output_labels_dir, filename)
                    shutil.move(filepath, destination_path)
                    print(f"'{filename}' 파일을 '{output_labels_dir}'(으)로 이동했습니다.")
                except Exception as e:
                    print(f"파일 이동 오류: {filepath} -> {destination_path} - {e}")

if __name__ == "__main__":
    # TODO: 여기에 YOLO 라벨 파일들이 있는 실제 입력 경로를 넣어주세요.
    input_directory = "./dataset/train3/labels/" 
    
    # TODO: 여기에 클래스 2 라벨 파일들이 이동될 출력 경로를 넣어주세요.
    output_directory = "./dataset/train3/제외/labels/"

    if os.path.exists(input_directory):
        move_labels_with_class_2(input_directory, output_directory)
        print("모든 라벨 파일 검사 및 이동이 완료되었습니다.")
    else:
        print(f"오류: 지정된 입력 경로 '{input_directory}'를 찾을 수 없습니다. 올바른 경로를 입력해주세요.")


'cam02_41502_42540.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42541.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42570.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42571.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42600.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42601.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42630.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42631.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42660.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42661.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42690.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42720.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42721.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42750.txt' 파일을 './dataset/train3/제외/labels/'(으)로 이동했습니다.
'cam02_41502_42751.t

### **클래스 분포도**

In [21]:
import os
from collections import defaultdict

def count_yolo_classes(labels_dir):
    """
    YOLO 라벨 파일(.txt)들을 읽고 각 클래스 번호별 라벨 개수를 계산합니다.

    Args:
        labels_dir (str): YOLO 라벨 파일들이 있는 디렉토리 경로.

    Returns:
        dict: 각 클래스 번호와 해당 클래스의 라벨 개수를 담은 딕셔너리.
    """
    class_counts = defaultdict(int) # 클래스별 개수를 저장할 딕셔너리

    if not os.path.isdir(labels_dir):
        print(f"오류: 지정된 경로 '{labels_dir}'는 유효한 디렉토리가 아닙니다.")
        return {}

    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_dir, filename)
            
            try:
                with open(filepath, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if parts: # 빈 줄이 아닌 경우
                            try:
                                class_number = int(parts[0])
                                class_counts[class_number] += 1
                            except ValueError:
                                # 첫 번째 부분이 숫자가 아닌 경우 (예: 잘못된 형식) 경고 출력
                                print(f"경고: '{filename}' 파일의 '{line.strip()}' 줄에서 유효하지 않은 클래스 번호가 감지되었습니다.")
            except Exception as e:
                print(f"파일 읽기 오류: {filepath} - {e}")
                continue
    
    return dict(class_counts)

if __name__ == "__main__":
    # TODO: 여기에 YOLO 라벨 파일들이 있는 실제 경로를 넣어주세요.
    # 예시: labels_directory = "C:/Users/user/Desktop/yolo_dataset/labels"
    labels_directory = "./dataset/train3/labels/" 
    
    if os.path.exists(labels_directory):
        print(f"'{labels_directory}' 경로의 라벨 파일들을 분석 중...")
        counts = count_yolo_classes(labels_directory)
        
        if counts:
            print("\n--- 클래스별 라벨 개수 ---")
            # 클래스 번호 순으로 정렬하여 출력
            for class_num in sorted(counts.keys()):
                print(f"Class {class_num}: {counts[class_num]}개")
            print("------------------------")
        else:
            print("해당 디렉토리에서 라벨 정보를 찾을 수 없거나 처리할 라벨이 없습니다.")
    else:
        print(f"오류: 지정된 경로 '{labels_directory}'를 찾을 수 없습니다. 올바른 경로를 입력해주세요.")


'./dataset/train3/labels/' 경로의 라벨 파일들을 분석 중...

--- 클래스별 라벨 개수 ---
Class 0: 14073개
Class 1: 13649개
Class 2: 9222개
Class 3: 8586개
------------------------
